In [2]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms


batch_size=200
learning_rate=0.01
epochs=10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnistdata', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnistdata', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)






class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.315370
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.075150
Train Epoch: 0 [40000/60000 (67%)]	Loss: 1.640469

Test set: Average loss: 0.0063, Accuracy: 6460/10000 (64%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.239280
Train Epoch: 1 [20000/60000 (33%)]	Loss: 1.110388
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.802636

Test set: Average loss: 0.0036, Accuracy: 8038/10000 (80%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.748133
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.577616
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.514141

Test set: Average loss: 0.0029, Accuracy: 8209/10000 (82%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.644341
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.586839
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.441489

Test set: Average loss: 0.0017, Accuracy: 9071/10000 (90%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.351269
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.285073
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.370143

Test set: Average loss: 0.0015